In [11]:
import json
import pandas as pd
from konlpy.tag import  Okt
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import DBSCAN
import numpy as np
import os

In [12]:
# 월별 데이터로 생성하기
json_list = os.listdir('../news-crawler/it/')
json_files = [file for file in json_list if file.endswith('.json')]  
data = []
df = pd.DataFrame()
for i in json_files:
    for line in open(('../news-crawler/it/'+i),"r", encoding='utf-8-sig'):
        df = pd.concat([df, pd.DataFrame(json.loads(line), columns=['id', 'title', 'content', 'date', 'like'])])  

In [13]:
# # 주차별 데이터로 생성하기
# with open("../news-crawler/it/2021-11-07-20.json", "r", encoding='utf-8-sig') as f:
#     tmp = json.load(f)
# df = pd.DataFrame(json.loads(line), columns=['id', 'title', 'content', 'date', 'like'])

In [14]:
okt = Okt() 
# okt.analyze  #구(Phrase) 분석
# okt.morphs   #형태소 분석
# okt.nouns    #명사 분석
# okt.pos      #형태소 분석 태깅

noun_list = []
for content in tqdm(df['content']): 
    nouns = okt.nouns(content)
    noun_list.append(nouns)
df['nouns'] = noun_list
print(df.head())

100%|██████████| 4732/4732 [03:15<00:00, 24.22it/s]

                                                  id  \
0  {'company': '008', 'id': '0004664095', 'sid': ...   
1  {'company': '030', 'id': '0002978817', 'sid': ...   
2  {'company': '030', 'id': '0002978812', 'sid': ...   
3  {'company': '081', 'id': '0003226105', 'sid': ...   
4  {'company': '243', 'id': '0000017876', 'sid': ...   

                                            title  \
0        '아이폰13' 닮았다는 갤럭시S22…1월 아닌 2월 출격설 왜?[IT썰]   
1  [이슈분석]기본도 안지킨 관리부실이 불러온 참사...제도개선, 보상안 논란 지속될듯   
2                       램리서치 한국 R&D센터, 150명 인력 채용   
3    [달콤한 사이언스] 우리집 반려견, 이름 부르면 갸우뚱 하는 이유 알고보니...   
4                    반도체 때문에 꺾인 스마트폰 시장…프리미엄이 이끈다   

                                             content                 date  \
0  원본보기\n갤럭시S22 렌더링 이미지. /사진=렛츠고디지털\n내년 1월에 출시됐던 ...  2021-10-31 16:05:50   
1  원본보기\n조경식 과기정통부 차관이 KT 네트워크장애사태 원인을 브리핑하고 있다.\...  2021-10-31 16:02:14   
2  세계 4대 반도체 장비 회사인 램리서치의 한국 연구개발(R&D)센터가 내년 2월 가...  2021-10-31 16:02:10   
3  반려견들 기억 되살리기 위해 집중할 때 고개 갸우뚱\n원본보

In [15]:
# 문서를 명사 집합으로 보고 문서 리스트로 치환 (tfidfVectorizer 인풋 형태를 맞추기 위해)
text = [" ".join(noun) for noun in df['nouns']]

tfidf_vectorizer = TfidfVectorizer(min_df = 5, ngram_range=(1,5))
tfidf_vectorizer.fit(text)
vector = tfidf_vectorizer.transform(text).toarray()

vector = np.array(vector) # Normalizer를 이용해 변환된 벡터
model = DBSCAN(eps=0.3,min_samples=6, metric = "cosine")
# 거리 계산 식으로는 Cosine distance를 이용
result = model.fit_predict(vector)
df['result'] = result

In [16]:
# Delete noise or garbage data
df = df[df['result'] != -1]
df = df[df['result'] != 0]

cluster_dict = {}

# Create clustering using titles
for i in df['result'].unique().tolist():
    cluster_dict[i] = df[df['result'] == i].title.tolist()

# Order by Hot topic    
sorted_cluster_dict = sorted(cluster_dict.items(), key=lambda x : len(x[1]), reverse=True)

In [31]:
final_dict = {}

# Create 10 data sets of clustered data
for idx, (cluster_num, titles) in enumerate(sorted_cluster_dict[:10]):
    keywords_list = []
    id_list = []
    like_list = []

    # Extract keyword from title
    for title in titles:
        keywords = okt.phrases(title)
        keywords_list.extend(keywords)
        # Add like / ID for sorting and linking
        id_list = df[df['result']==cluster_num].id.tolist()
        like_list = df[df['result']==cluster_num].like.tolist()
    # Create keyword standard
    standard_keyword = set(keywords_list)

    tmp_keyword = []

    # Keyword Counting
    for keyword in standard_keyword:
        tmp_keyword.append((keyword, keywords_list.count(keyword)))

    # Hot Keyword Top 3
    tmp_keyword.sort(key = lambda x : x[1], reverse=True)
    final_keyword = [keyword for keyword in tmp_keyword[:5]]
    print(final_keyword)
    # Create dictonary file
    final_data = []
    for id, title, like in zip(id_list, titles, like_list):
        id['like'] = like
        id['title'] = title
        final_data.append(id)
    final_dict[f"cluster{idx}"] = {}
    final_dict[f'cluster{idx}']['keyword'] = final_keyword
    final_dict[f'cluster{idx}']['data'] = final_data

[('가상', 17), ('자산', 17), ('과세', 16), ('가상자산', 12), ('유예', 10)]
[('카카오', 14), ('게임', 14), ('오딘', 13), ('카카오게임즈', 13), ('개발', 10)]
[('스팸', 11), ('불법', 11), ('은행', 7), ('정부', 6), ('사칭', 5)]
[('누리', 11), ('영상', 10), ('공개', 7), ('누리호가', 6), ('호가', 6)]
[('워치', 12), ('골프', 11), ('PXG', 11), ('삼성', 10), ('에디', 10)]
[('와이파이', 10), ('5G', 9), ('버스', 8), ('5G로', 5), ('버스 와이파이', 4)]
[('카카오', 11), ('아이돌', 9), ('넷마블', 7), ('가상', 6), ('메타', 6)]
[('금성', 11), ('8일', 7), ('9년', 4), ('8일 오후', 4), ('오후', 4)]
[('누리', 11), ('누리호', 10), ('산화제', 7), ('탱크', 7), ('압력', 7)]
[('AI', 9), ('KT', 9), ('구현모', 8), ('일상', 7), ('24시간', 4)]


In [30]:
final_dict

{'cluster0': {'keyword': [('가상', 17),
   ('자산', 17),
   ('과세', 16),
   ('가상자산', 12),
   ('유예', 10)],
  'data': [{'company': '018',
    'id': '0005077505',
    'sid': '105',
    'like': 54,
    'title': '이재명·민주당 한 목소리 “내년 가상자산 과세 유예해야”'},
   {'company': '014',
    'id': '0004733906',
    'sid': '105',
    'like': 0,
    'title': '선거 앞두고… 與 "정부 설득해 가상자산 과세 유예할 것"'},
   {'company': '092',
    'id': '0002238133',
    'sid': '105',
    'like': 0,
    'title': '민주당, 가상자산 과세유예 당론으로 채택하나'},
   {'company': '138',
    'id': '0002112896',
    'sid': '105',
    'like': 0,
    'title': '내년 1월 시행 예정 가상자산 과세에 전문가들 "준비 부족" 우려'},
   {'company': '293',
    'id': '0000036838',
    'sid': '105',
    'like': 43,
    'title': "두 달 후 가상자산 과세한다는 정부, 어떻게 할 건지 '깜깜이'"},
   {'company': '018',
    'id': '0005077204',
    'sid': '105',
    'like': 0,
    'title': '돈 잃고도 양도세?…내년 가상자산 과세 허점 많아'},
   {'company': '014',
    'id': '0004733710',
    'sid': '105',
    'like': 0,
    'title': '대선 앞둔 민주당 "가상자산 과세유예" 목소리 높인다